In [2]:
################################################################################
# 6 Multiple Resource Theory
# ----------------------------
# where's reference from?
# Multiple Resource Theory is to predict the relative differences in task
# interference between different task configurations.
#
# parameters shorthand explainations for task analysis shell and conflict matrix:
#       Task Analysis Shell: PC = Perception&Cognition, R = Responding,
#                           V (under Modalities) = Visual, A = Auditory,
#                           V (under Codes) = Verbal, S = Spatial
#       Conflict Matrix: A and V in the first position = Auditory and Visual
#                       V and S in the second position = Verbal and spatial
#                       C = Cognitive, R = Response
# Model limitations:
# 1) assuming all tasks are alike
################################################################################

import tkinter as tk
from tkinter import ttk
import numpy as np
from itertools import product, combinations, combinations_with_replacement, permutations
import matplotlib
matplotlib.use("TkAgg")                         # for 3d cube visualization
import matplotlib.backends.backend_tkagg as tkagg
from mpl_toolkits.mplot3d import Axes3D, art3d
import matplotlib.pyplot as plt

class MRT(tk.Frame):
    tasks = ['driving', 'reading', 'listening']
    taskAmount = 3

    def __init__(self, parent=None):
        tk.Frame.__init__(self, parent)
        self.pack()
        self.makeWidgets()
        self.defineTasks()
        self.flagTaskSelected = [0 for x in range(self.taskAmount)] # flags of selected tasks
        for x in range(self.taskAmount):
            self.flagTaskSelected[x] = False
        self.counterTaskSelected = 0                                # to limit maximum two tasks are selected
        self.analysisTasks()
        self.visualizeCube()
        self.computeConflictMatrix()

    # GUI layout
    def makeWidgets(self):
        self.frameTaskSelection = tk.Canvas(self)
        self.frameSign_taskSelection_cube = tk.Frame(self)
        self.frameCube = tk.Frame(self)
        self.frameTaskAnalysisShell = tk.Frame(self)
        self.frameConflictMatrix = tk.Frame(self)
        self.frameInterferenceValue = tk.Frame(self)
        self.frameTaskSelection.grid(row=0, column=0, padx=10, pady=10)
        self.frameSign_taskSelection_cube.grid(row=0, column=1, pady=10)
        self.frameCube.grid(row=0, column=2, padx=10, pady=10)
        ttk.Separator(self, orient=tk.HORIZONTAL).grid(row=1, column=0, columnspan=3, sticky=tk.EW, padx=10)
        self.frameTaskAnalysisShell.grid(row=2, column=0, columnspan=2, padx=10, pady=10)
        self.frameConflictMatrix.grid(row=2, column=2, padx=10, pady=10)
        self.frameInterferenceValue.grid(row=2, column=3, padx=10, pady=10)
        self.sign_taskSelection_cube = tk.Button(self.frameSign_taskSelection_cube, text='==>', width=20)
        self.sign_taskSelection_cube.grid(row=0, column=0)
        self.sign_taskSelection_cube.bind('<ButtonRelease-1>', self.onSign_taskSelection_cubeClicked)

    # define arrows indicating operation flow
    def defineSigns(self):
        # self.sign_taskAnalysisShell_conflictMatrix
        # self.sign_conflictMatrix_InterferenceValue
        print ("define arrows")

    # define tasks
    def defineTasks(self):
        taskBrick_positionX = 5
        taskBrick_positionY = 5
        taskBrick_width = 200
        taskBrick_height = 50
        fontSize = 16
        self.taskBrick = [0 for x in range(self.taskAmount)]
        self.taskBrickText = [0 for x in range(self.taskAmount)]
        for x in range(3):
            self.taskBrick[x] = self.frameTaskSelection.create_rectangle(taskBrick_positionX, taskBrick_positionY+x*taskBrick_height, taskBrick_positionX+taskBrick_width, taskBrick_positionY+taskBrick_height+x*taskBrick_height, fill='#63B2A9', tags=self.tasks[x])
            self.taskBrickText[x] = self.frameTaskSelection.create_text(taskBrick_positionX + 0.5*taskBrick_width, taskBrick_positionY+0.5*taskBrick_height+x*taskBrick_height, text=self.tasks[x], tags=self.tasks[x], font=("Arial", fontSize))
            self.frameTaskSelection.tag_bind(self.tasks[x], '<ButtonPress-1>', self.selectTasks)

    # detect task selection
    def selectTasks(self, event):
        eventObj = self.frameTaskSelection.find_closest(event.x, event.y)
        eventObjTag = self.frameTaskSelection.itemcget(eventObj, 'tags')
        for i in range(self.taskAmount):
            if eventObjTag == self.tasks[i] + ' current':
                if self.flagTaskSelected[i] == True:
                    self.flagTaskSelected[i] = False
                    self.counterTaskSelected -= 1
                    self.frameTaskSelection.itemconfigure(self.taskBrick[i], fill='#63B2A9')
                    self.frameTaskSelection.move(self.tasks[i], -30, 0)
                elif self.flagTaskSelected[i] == False and self.counterTaskSelected < 2:
                    self.flagTaskSelected[i] = True
                    self.counterTaskSelected += 1
                    self.frameTaskSelection.itemconfigure(self.taskBrick[i], fill='#C0FFF8')
                    self.frameTaskSelection.move(self.tasks[i], 30, 0)
        # output self.flagTaskSelected[] indicating selected two tasks

# task analysis shell
    def analysisTasks(self):
        self.tableTaskAnalysisShell = ttk.Treeview(self.frameTaskAnalysisShell, columns=('codes', 'stages', 'modalities'), selectmode = "browse")
        self.labelTaskAnalysisTitle = tk.Label(self.frameTaskAnalysisShell, text="Task Analysis Shell", anchor=tk.CENTER)
        self.tableTaskAnalysisShell.grid(row=0, column=0)
        self.labelTaskAnalysisTitle.grid(row=1, column=0)
        self.tableTaskAnalysisShell.column('#0', width=80)
        self.tableTaskAnalysisShell.column('codes', width=70, stretch=True)
        self.tableTaskAnalysisShell.column('stages', width=70, stretch=True)
        self.tableTaskAnalysisShell.column('modalities', width=70, stretch=True)
        self.tableTaskAnalysisShell.heading('codes', text='Codes')
        self.tableTaskAnalysisShell.heading('stages', text='Stages')
        self.tableTaskAnalysisShell.heading('modalities', text='Modalities')
        self.tableTaskAnalysisShell.insert('', 0, 'driving', text=self.tasks[0], tags=0, values='S PC+R V')
        self.tableTaskAnalysisShell.insert('', 1, 'reading', text=self.tasks[1], tags=1, values='V PC V')
        self.tableTaskAnalysisShell.insert('', 2, 'listening', text=self.tasks[2], tags=2, values='V PC A')
        self.taskAnalysisValues = [[[1, 0], [1, 1], [0, 1]], [[0, 1], [1, 0], [0, 1]], [[0, 1], [1, 0], [1, 0]]]     # in terms of taskAnalysisValues[task][dimension][level]

# 3D Cube visualization
    def visualizeCube(self):
        # embed matplotlib 3D plotting in tkinter GUI
        self.fig = plt.Figure(figsize=(4, 3), dpi=100)
        self.canvas = tkagg.FigureCanvasTkAgg(self.fig, master=self.frameCube)
        self.ax = self.fig.add_subplot(111, projection='3d')
        self.ax.set_aspect("equal")
        # plot surfaces of the cube
        o = [[0, 1], [0, -1]]
        p = [[0, 1], [0, -1]]
        self.mesh1 = [0 for z in range(4)]
        self.mesh2 = [0 for z in range(4)]
        productResult = list(product(o, p))
        for z in range(4):
            self.mesh1[z], self.mesh2[z] = np.meshgrid(productResult[z][0], productResult[z][1])
        self.surface = [[[0 for z in range(4)] for y in range(3)] for x in range(3)]
        listPlotParam = [0 for y in range(3)]
        for y in range(3):
            for z in range(4):
                listPlotParam[y] = [-1+y, self.mesh1[z], self.mesh2[z]]
                # surface[XYZ][-101][Quadrant]
                self.surface[0][y][z] = self.ax.plot_surface(listPlotParam[y][0], listPlotParam[y][1], listPlotParam[y][2], alpha=0.2, color='#C0FFF8')
                self.surface[1][y][z] = self.ax.plot_surface(listPlotParam[y][1], listPlotParam[y][0], listPlotParam[y][2], alpha=0.2, color='#C0FFF8')
                self.surface[2][y][z] = self.ax.plot_surface(listPlotParam[y][1], listPlotParam[y][2], listPlotParam[y][0], alpha=0.2, color='#C0FFF8')
        # plot edges of the cube
        q = [-1, 1]
        for m, n in combinations(np.array(list(product(q, q, q))), 2):
            if np.sum(np.abs(m-n)) == q[1]-q[0]:
                self.ax.plot3D(*zip(m, n), color='white')
        # labelling dimensions and associated levels
        labelAxisText = ['Codes', 'Stages', 'Modalities']           # X, Y, Z axis respectively
        labelLevelsText = [['Sptial', 'Verbal'], ['Perception&\nCognition', 'Responding'], ['Auditory', 'Visual']]
        self.ax.set_xlabel(labelAxisText[0])
        self.ax.set_ylabel(labelAxisText[1])
        self.ax.set_zlabel(labelAxisText[2])
        axisTicks = np.arange(-1, 1.1, 0.5)
        self.ax.set_xticks(axisTicks)
        self.ax.set_yticks(axisTicks)
        self.ax.set_zticks(axisTicks)
        labelAxis = [0 for a in range(3)]
        for a in range(3):
            labelAxis[a] = [item.get_text() for item in self.ax.get_xticklabels()]
            labelAxis[a][1] = labelLevelsText[a][0]
            labelAxis[a][3] = labelLevelsText[a][1]
        self.ax.set_xticklabels(labelAxis[0])
        self.ax.set_yticklabels(labelAxis[1])
        self.ax.set_zticklabels(labelAxis[2])
        # wanna hide grid of coordinate system
        # ax.tick_params(axis='x', length=0.0, width=0.0)
        self.canvas.show()
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        # # surface[0][0][0].remove()
        # # surface[0][0][0] = ax.plot_surface(listPlotParam[0][0], listPlotParam[0][1], listPlotParam[0][2], alpha=0.2, color='red')


    # make a table of conflict matrix, with real-time slot marking
    def computeConflictMatrix(self):
        self.tableConflictMatrix = tk.Canvas(self.frameConflictMatrix, relief=tk.GROOVE, bd=2, bg='white')
        self.tableConflictMatrix.grid(row=0, column=0)
        tk.Label(self.frameConflictMatrix, text="Conflict Matrix").grid(row=1, column=1)
        cellWidth = 40
        cellHeight = 25
        self.cell = [[0 for y in range(9)] for x in range(10)]
        for x in range(10):
            for y in range(9):
                self.cell[x][y] = self.tableConflictMatrix.create_rectangle((x-1)*cellWidth, (y-1)*cellHeight, x*cellWidth, y*cellHeight, outline='')
        self.textPerceptual = self.tableConflictMatrix.create_text(3*cellWidth, 0.5*cellHeight, text='Perceptual')
        self.textCognitive = self.tableConflictMatrix.create_text(6*cellWidth, 0.5*cellHeight, text='Cognitive')
        self.textResponse = self.tableConflictMatrix.create_text(8*cellWidth, 0.5*cellHeight, text='Response')
        levelsOfDimensions = ['VS', 'VV', 'AS', 'AV', 'CS', 'CV', 'RS', 'RV']
        self.textLevelsOfDimensions = [0 for m in range(2)]
        for m in range(2):
            self.textLevelsOfDimensions[m] = [0 for n in range(8)]
            for n in range(8):
                if m == 0:
                    self.textLevelsOfDimensions[m][n] = self.tableConflictMatrix.create_text(0.5*cellWidth, (2.5+n)*cellHeight, text=levelsOfDimensions[n])
                elif m == 1:
                    self.textLevelsOfDimensions[m][n] = self.tableConflictMatrix.create_text((1.5+n)*cellWidth, 1.5*cellHeight, text=levelsOfDimensions[n])
        valuesOfConflicts = ['0.8', '0.6', '0.6', '0.4', '0.7', '0.5', '0.4', '0.2',
                            '', '0.8', '0.4', '0.6', '0.5', '0.7', '0.2', '0.4',
                            ' ', ' ', '0.8', '0.4', '0.7', '0.5', '0.4', '0.2',
                            ' ', ' ', '0', '0.8', '0.5', '0.7', '0.2', '0.4',
                            ' ', ' ', '0', ' ', '0.8', '0.6', '0.6', '0.4',
                            ' ', ' ', ' ', ' ', ' ', '0.8', '0.4', '0.6',
                            ' ', ' ', ' ', ' ', ' ', ' ', '0.8', '0.6',
                            ' ', ' ', ' ', ' ', ' ', ' ', '0.6', '1.0']
        self.textValuesOfConflicts = [[0 for q in range(8)] for p in range(8)]
        for p in range(8):
            for q in range(8):
                self.textValuesOfConflicts[p][q] = self.tableConflictMatrix.create_text((1.5+q)*cellWidth, (2.5+p)*cellHeight, text=valuesOfConflicts[p*8+q])
        self.tableConflictMatrix.create_line(cellWidth, cellHeight, 9*cellWidth, cellHeight, width=2)
        self.tableConflictMatrix.create_line(10, 6*cellHeight, 9*cellWidth, 6*cellHeight, dash=(4, 4))
        self.tableConflictMatrix.create_line(10, 8*cellHeight, 9*cellWidth, 8*cellHeight)
        self.tableConflictMatrix.create_line(5*cellWidth, 10, 5*cellWidth, 10*cellHeight, dash=(4, 4))
        self.tableConflictMatrix.create_line(7*cellWidth, 10, 7*cellWidth, 10*cellHeight)
        self.tableConflictMatrix.itemconfigure(self.cell[7][5], fill='#C0FFF8')

    def onSign_taskSelection_cubeClicked(self, event):
        # find which two tasks are selected
        self.indexTaskSelected = []
        self.surfaceTaskSelected = []           # self.surfaceTaskSelected[][] = [[XYZ/dimension, -101/level], [XYZ/dimension, -101/level] 
        self.surfaceMarked = []                 # self.surfaceMarked[][] = [[XYZ/dimension, -101/level, Quadrant], [XYZ/dimension, -101/level, Quadrant]]
        flagBothLevels = []
        for x in range(self.taskAmount):
            if self.flagTaskSelected[x] == True:
                self.indexTaskSelected.append(x)
                # mark associated line in task analysis shell
                self.tableTaskAnalysisShell.tag_configure(x, background='#C0FFF8')
                # mark associated sub-cube in 3d cube visualization
                for a in range(3):
                    counter = 0
                    for b in range(2):
                        if self.taskAnalysisValues[x][a][b]:
                            self.surfaceTaskSelected.append([a, b])
                            self.surfaceMarked.append([a, b, ''])
                            self.surfaceMarked.append([a, b+1, ''])
                            counter += 1
                    if counter == 2:
                        flagBothLevels.append(True)
                    else:
                        flagBothLevels.append(False)
                print (self.surfaceTaskSelected)
                listLevelsProductEntry = [[], [], []]
                counter = 0
                for a in range(3):
                    listLevelsProductEntry[a].append(self.surfaceTaskSelected[counter])
                    counter += 1
                    if flagBothLevels[a] == True:
                        listLevelsProductEntry[a].append(self.surfaceTaskSelected[counter])
                        counter += 1
                listLevelsProduct = list(product(listLevelsProductEntry[0], listLevelsProductEntry[1], listLevelsProductEntry[2]))
                print (listLevelsProduct)


                for a in range(len(listLevelsProduct)):
                    print ("a=%d" %a)
                    for b in range(3):
                        if listLevelsProduct[a][b][1] - listLevelsProduct[a][b-1][1] == 0:
                            if listLevelsProduct[a][b][1] == 0:
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1], 3])
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1]+1, 3])
                                # self.surfaceMarked[2*(b-2)][2] = 3
                                # self.surfaceMarked[2*(b-2)+1][2] = 3
                                print ("3")
                            else:
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1], 0])
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1]+1, 0])
                                # self.surfaceMarked[2*(b-2)][2] = 0
                                # self.surfaceMarked[2*(b-2)+1][2] = 0
                                print ("0")
                        elif listLevelsProduct[a][b][1] - listLevelsProduct[a][b-1][1] == 1:
                            if b == 0:
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1], 1])
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1]+1, 1])
                                # self.surfaceMarked[2*(b-2)][2] = 1
                                # self.surfaceMarked[2*(b-2)+1][2] = 1
                            else:
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1], 2])
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1]+1, 2])
                                # self.surfaceMarked[2*(b-2)][2] = 2
                                # self.surfaceMarked[2*(b-2)+1][2] = 2
                            print ("2")
                        elif listLevelsProduct[a][b][1] - listLevelsProduct[a][b-1][1] == -1:
                            if b == 0:
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1], 2])
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1]+1, 2])
                                # self.surfaceMarked[2*(b-2)][2] = 2
                                # self.surfaceMarked[2*(b-2)+1][2] = 2
                            else:
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1], 1])
                                self.surfaceMarked.append([(b+1)%3, self.surfaceTaskSelected[(b+1)%3][1]+1, 1])
                                # self.surfaceMarked[2*(b-2)][2] = 1
                                # self.surfaceMarked[2*(b-2)+1][2] = 1
                            print ("1")
                print (self.surfaceMarked)
                print (len(self.surfaceMarked))

                # for a in range(len(self.surfaceMarked)):
                #     self.surface[self.surfaceMarked[a][0]][self.surfaceMarked[a][1]][self.surfaceMarked[a][2]].remove()
                #     if self.surfaceMarked[a][0] == 0:
                #         self.surface[self.surfaceMarked[a][0]][self.surfaceMarked[a][1]][self.surfaceMarked[a][2]] = self.ax.plot_surface(self.surfaceMarked[a][1]-1, self.mesh1[self.surfaceMarked[a][2]], self.mesh2[self.surfaceMarked[a][2]], alpha=0.5, color='red')
                #     elif self.surfaceMarked[a][0] == 1:
                #         self.surface[self.surfaceMarked[a][0]][self.surfaceMarked[a][1]][self.surfaceMarked[a][2]] = self.ax.plot_surface(self.mesh1[self.surfaceMarked[a][2]], self.surfaceMarked[a][1]-1, self.mesh2[self.surfaceMarked[a][2]], alpha=0.5, color='red')
                #     elif self.surfaceMarked[a][0] == 2:
                #         self.surface[self.surfaceMarked[a][0]][self.surfaceMarked[a][1]][self.surfaceMarked[a][2]] = self.ax.plot_surface(self.mesh1[self.surfaceMarked[a][2]], self.mesh2[self.surfaceMarked[a][2]], self.surfaceMarked[a][1]-1, alpha=0.5, color='red')
                self.canvas.show()

                # mark associated cell in conflict matrix
            else:
                self.tableTaskAnalysisShell.tag_configure(x, background='white')

        # compute interference value -- define a function

# computational formula
    def computeInterferenceValue(self):
        print ("computational formula")
        # demand component = summing the average demand of both tasks
        # conflict component =

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Multiple Resource Theory")
    root.geometry("1000x600")
    root.resizable(width=False, height=False)
    MRT(root).mainloop()


[[0, 0], [1, 0], [1, 1], [2, 1]]
[([0, 0], [1, 0], [2, 1]), ([0, 0], [1, 1], [2, 1])]
a=0
1
3
2
a=1
1
2
0
[[0, 0, ''], [0, 1, ''], [1, 0, ''], [1, 1, ''], [1, 1, ''], [1, 2, ''], [2, 1, ''], [2, 2, ''], [1, 0, 2], [1, 1, 2], [2, 1, 3], [2, 2, 3], [0, 0, 2], [0, 1, 2], [1, 0, 2], [1, 1, 2], [2, 1, 2], [2, 2, 2], [0, 0, 0], [0, 1, 0]]
20
